In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from dataset import MELDDataset, Utterance, Dialogue
import pickle
from dummy_model import DummyModel
from torch.utils.data import DataLoader
from models.config import Config
from models.dialogue_gcn import DialogueGCN
from sklearn.metrics import f1_score, confusion_matrix



## Load data

In [11]:
config = Config()
audio_embed_path_test = "../MELD.Raw/audio_embeddings_feature_selection_emotion.pkl"
_, _, test_audio_emb = pickle.load(open(audio_embed_path_test, 'rb'))
test_dataset = MELDDataset("../MELD.Raw/test_sent_emo.csv", "../MELD.Raw/output_repeated_splits_test", test_audio_emb, name="test", config=config)
test_loader = DataLoader(test_dataset, batch_size=1)


{'joy': 0, 'anger': 1, 'disgust': 2, 'fear': 3, 'sadness': 4, 'neutral': 5, 'surprise': 6}


## Load model

In [12]:
model_path = "model_saves/text0_epoch_image_only_7.pt"
model = DialogueGCN(config)
model = model.to("cuda")

state_dict = torch.load(model_path)
model.load_state_dict(state_dict['model_state_dict'])
model.eval()

DialogueGCN(
  (text_encoder): GRU(768, 100, batch_first=True, bidirectional=True)
  (context_encoder): GRU(200, 100, batch_first=True, bidirectional=True)
  (pred_rel_l1): GraphConvolution()
  (suc_rel_l1): GraphConvolution()
  (same_speak_rel_l1): GraphConvolution()
  (diff_speak_rel_l1): GraphConvolution()
  (pred_rel_l2): GraphConvolution()
  (suc_rel_l2): GraphConvolution()
  (same_speak_rel_l2): GraphConvolution()
  (diff_speak_rel_l2): GraphConvolution()
  (edge_att_weights): Linear(in_features=200, out_features=200, bias=False)
  (text_attn): Linear(in_features=200, out_features=1, bias=True)
  (w_sentiment): Linear(in_features=400, out_features=3, bias=True)
  (w_emotion_1): Linear(in_features=400, out_features=200, bias=True)
  (w_emotion_2): Linear(in_features=200, out_features=7, bias=True)
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_em

In [3]:
legend = {0:'joy', 1:'anger', 2:'disgust', 3:'fear', 4:'sadness', 5:'neutral', 6:'surprise'}


In [4]:
def get_accuracy(predicted_emotion, predicted_sentiment, target):
    emotion_accuracy_acc = torch.eq(predicted_emotion, target[:,0]).sum().item() / target.size(0)
    sentiment_accuracy_acc = torch.eq(predicted_sentiment, target[:,1]).sum().item() / target.size(0)
    return emotion_accuracy_acc, sentiment_accuracy_acc


def test_model(model_name, model, test_loader):
    print("Testing " + model_name)
    model = model.eval()
    emotion_predicted_labels = []
    sentiment_predicted_labels = []
    emotion_target_labels = []
    sentiment_target_labels = []
    for i, (test_batch_input, test_batch_labels) in enumerate(test_loader):
        batch_emotion_correct_predicted_labels, batch_sentiment_predicted_labels, batch_val_count = validate_step(model, test_batch_input, test_batch_labels)
        emotion_predicted_labels.append(batch_emotion_correct_predicted_labels)
        #print(batch_emotion_correct_predicted_labels)
        sentiment_predicted_labels.append(batch_sentiment_predicted_labels)
        emotion_target_labels.append(torch.cat(test_batch_labels[0],0))
        sentiment_target_labels.append(torch.cat(test_batch_labels[1],0))

    emotion_predicted_labels = torch.cat(emotion_predicted_labels, 0).cuda()
    sentiment_predicted_labels = torch.cat(sentiment_predicted_labels, 0).cuda()
    emotion_target_labels = torch.cat(emotion_target_labels, 0)
    sentiment_target_labels = torch.cat(sentiment_target_labels, 0)
    target_labels = torch.cat([emotion_target_labels.unsqueeze(1), sentiment_target_labels.unsqueeze(1)], 1).cuda()

    emotion_f1_score = f1_score(emotion_target_labels.cpu().numpy(), emotion_predicted_labels.cpu().numpy(), average='weighted')        
    confusion = confusion_matrix(emotion_target_labels.cpu().numpy(), emotion_predicted_labels.cpu().numpy())
    emotion_accuracy, sentiment_accuracy = get_accuracy(emotion_predicted_labels, sentiment_predicted_labels, target_labels)
    #emotion_recalls, sentiment_recalls = get_recall_for_each_class(emotion_predicted_labels, sentiment_predicted_labels, target_labels)
    #emotion_precisions, sentiment_precisions = get_precision_for_each_class(emotion_predicted_labels, sentiment_predicted_labels, target_labels)
    #emotion_f1s, sentiment_f1s = get_f1_score_for_each_class(emotion_precisions, emotion_recalls, sentiment_precisions, sentiment_recalls)
    #emotion_weighted_f1, sentiment_weighted_f1 = get_weighted_F1(emotion_f1s, sentiment_f1s, target_labels)

    print("Validation Accuracy (Emotion): ", emotion_accuracy)
    print("F1 Weighted", emotion_f1_score)
    print("Confusion matrix", confusion)

def validate_step(model, input, target):
    target = torch.LongTensor(target).to("cuda")
    (output_logits_emotion, output_logits_sentiment) = model(input)
    output_labels_emotion = torch.argmax(output_logits_emotion, dim=1)
    output_labels_sentiment = torch.argmax(output_logits_sentiment, dim=1)
    return output_labels_emotion, output_labels_sentiment, target[0].size()

def preprocess_datapoint(datapoint):
    datapoint = list(datapoint[0])
    text = datapoint[0]
    for i, utt in enumerate(text):
        text[i] = (utt,)
    datapoint[0] = text    
    return datapoint

In [6]:
test_model("GCN_TEXT model", model, test_loader)

Testing GCN_TEXT model
Validation Accuracy (Emotion):  0.6030651340996168
F1 Weighted 0.5775735176864728
Confusion matrix [[ 211   32    1    0    5  107   46]
 [  38  141    6    1    9  102   48]
 [   6   14    1    0    5   31   11]
 [   4    8    0    3    5   22    8]
 [  21   24    0    1   29  111   22]
 [  81   25    3    2   24 1022   99]
 [  34   26    0    0    2   52  167]]


## Friends (test set of our dataset)

In [16]:
dialogue_id = 30
datapoint = preprocess_datapoint(test_dataset[dialogue_id])
labels = test_dataset[dialogue_id][1]
predictions = validate_step(model, datapoint, labels)[0]

for i, pred in enumerate(predictions):
    print(legend[int(pred)], " : ", test_dataset[dialogue_id][0][0][i])


surprise  :  Oh my God, what happened?
surprise  :  Oh. God, crazy Chandler. He spun me...off...the...bed!
joy  :  Wow! Spinning that sounds like fun.
neutral  :  Oh,  I wish. No, you know he was just trying Ross's Hug and Roll thing.
surprise  :  Ross's what?
disgust  :  You know what, where he hugs you and kinda rolls you away and... Oh... my....God.


## Seinfeld (test set of our dataset)

In [28]:
texts = [(0, "maybe she just likes me for me."),
(1, "you want these?"),
(2, "what?"),
(1, "I stink!"),
(1, "I can’t play!"), 
(1, "The ball is just sitting there and I can’t hit it!"),
(1, "I only hit one really good ball that went way out!"),
(1, "I have no concentration!"),
(2, "What’s wrong?"),
(1, "I can’t get rid of the sand."),
(1, "There is still some in here won’t go away."),
(1, "I even got sand in the pockets."),
(2, "Come on, you are getting that all over the floor.")
]
utterances = []
for i, text_speaker in enumerate(texts):
    speaker, text = text_speaker
    utterance = Utterance(0, i, text, speaker, 0, 0, 0, 0, 0)
    utterances.append(utterance)
dialogue = Dialogue(0, utterances, visual_features=False)

In [29]:
datapoint = preprocess_datapoint(dialogue.get_data())

https://www.youtube.com/watch?v=iBaPplviA8o

In [30]:
for i, pred in enumerate(validate_step(model, datapoint, [0])[0]):
    print(legend[int(pred)], " : ", texts[i][1])


neutral  :  maybe she just likes me for me.
surprise  :  you want these?
surprise  :  what?
joy  :  I stink!
anger  :  I can’t play!
anger  :  The ball is just sitting there and I can’t hit it!
anger  :  I only hit one really good ball that went way out!
anger  :  I have no concentration!
neutral  :  What’s wrong?
neutral  :  I can’t get rid of the sand.
neutral  :  There is still some in here won’t go away.
neutral  :  I even got sand in the pockets.
neutral  :  Come on, you are getting that all over the floor.
